# External Data Tokyo

EO: https://apps.sentinel-hub.com/eo-browser/?zoom=12&lat=35.6529&lng=139.6885&themeId=DEFAULT-THEME

In [1]:
import os
import warnings
import random
from pprint import pprint
import copy
from typing import List, Tuple
import glob
import json
import csv
# import dataclasses
from joblib import Parallel, delayed
from typing import List, Set, Dict, Any

from tqdm import tqdm
import numpy as np
import pandas as pd
from box import Box, from_file
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from scipy.optimize import minimize
from sklearn.metrics import jaccard_score
import cv2
import seaborn as sns

In [2]:
class CFG(object):
    # basic
    debug: bool = False
    folds: int  = 4
    seed: int   = 417
    eps: float  = 1e-12
    
    # data
    PATH_CSV: str  =  f'../data/019_Tokyo/019_seed{seed}.csv'
    PATH_NPY_ROOT = f'../data/019_Tokyo/'
    PATH_DATA = '../data/tokyo/'
    os.makedirs(PATH_DATA, exist_ok=True), os.makedirs(PATH_NPY_ROOT + 'img/', exist_ok=True)
    
# box
cfg = Box({k:v for k, v in dict(vars(CFG)).items() if '__' not in k})
    
# 乱数のシードを設定
np.random.seed(cfg.seed)
random.seed(cfg.seed)
    
pprint(cfg)

{'PATH_CSV': '../data/019_Tokyo/019_seed417.csv',
 'PATH_DATA': '../data/tokyo/',
 'PATH_NPY_ROOT': '../data/019_Tokyo/',
 'debug': False,
 'eps': 1e-12,
 'folds': 4,
 'seed': 417}


## CLI

In [3]:
!aws --version

aws-cli/1.27.131 Python/3.8.16 Linux/5.15.0-69-generic botocore/1.29.131


## Download

`2020/04/30` - `2023/05/30` 

few clous scene

In [4]:
PATHS_SENTILEL2_S3 = [
    "s3://sentinel-s2-l2a/tiles/54/S/UE/2023/5/17/0/",
    "s3://sentinel-s2-l2a/tiles/54/S/UE/2023/5/2/0/",
    "s3://sentinel-s2-l2a/tiles/54/S/UE/2023/4/27/0/",
    "s3://sentinel-s2-l2a/tiles/54/S/UE/2023/4/17/0/",
    "s3://sentinel-s2-l2a/tiles/54/S/UE/2023/4/12/0/",
    "s3://sentinel-s2-l2a/tiles/54/S/UE/2023/3/8/0/",
    "s3://sentinel-s2-l2a/tiles/54/S/UE/2023/2/26/0/",
    "s3://sentinel-s2-l2a/tiles/54/S/UE/2023/2/21/0/",
    "s3://sentinel-s2-l2a/tiles/54/S/UE/2023/2/16/0/",
    "s3://sentinel-s2-l2a/tiles/54/S/UE/2023/2/6/0/",
]

In [5]:
for PATH_S3 in tqdm(PATHS_SENTILEL2_S3):
    SCENE_ID = PATH_S3.split('tiles')[1].replace('/', '-')[1:-3]
    print(SCENE_ID)
    OUTPUT_PATH = f'{cfg.PATH_DATA}/{SCENE_ID}/'
    os.makedirs(OUTPUT_PATH, exist_ok=True)
    
    !aws s3 cp {PATH_S3} {OUTPUT_PATH} --recursive --request-payer
    # break

  0%|          | 0/10 [00:00<?, ?it/s]

54-S-UE-2023-5-17
download: s3://sentinel-s2-l2a/tiles/54/S/UE/2023/5/17/0/R10m/AOT.jp2 to ../data/tokyo/54-S-UE-2023-5-17/R10m/AOT.jp2
download: s3://sentinel-s2-l2a/tiles/54/S/UE/2023/5/17/0/R10m/B03.jp2 to ../data/tokyo/54-S-UE-2023-5-17/R10m/B03.jp2
download: s3://sentinel-s2-l2a/tiles/54/S/UE/2023/5/17/0/R10m/B04.jp2 to ../data/tokyo/54-S-UE-2023-5-17/R10m/B04.jp2
download: s3://sentinel-s2-l2a/tiles/54/S/UE/2023/5/17/0/R20m/AOT.jp2 to ../data/tokyo/54-S-UE-2023-5-17/R20m/AOT.jp2
download: s3://sentinel-s2-l2a/tiles/54/S/UE/2023/5/17/0/R10m/B08.jp2 to ../data/tokyo/54-S-UE-2023-5-17/R10m/B08.jp2
download: s3://sentinel-s2-l2a/tiles/54/S/UE/2023/5/17/0/R10m/B02.jp2 to ../data/tokyo/54-S-UE-2023-5-17/R10m/B02.jp2
download: s3://sentinel-s2-l2a/tiles/54/S/UE/2023/5/17/0/R10m/TCI.jp2 to ../data/tokyo/54-S-UE-2023-5-17/R10m/TCI.jp2
download: s3://sentinel-s2-l2a/tiles/54/S/UE/2023/5/17/0/R20m/B01.jp2 to ../data/tokyo/54-S-UE-2023-5-17/R20m/B01.jp2
download: s3://sentinel-s2-l2a/tiles/5

 10%|█         | 1/10 [04:45<42:45, 285.00s/it]

54-S-UE-2023-5-2
download: s3://sentinel-s2-l2a/tiles/54/S/UE/2023/5/2/0/R10m/AOT.jp2 to ../data/tokyo/54-S-UE-2023-5-2/R10m/AOT.jp2
download: s3://sentinel-s2-l2a/tiles/54/S/UE/2023/5/2/0/R10m/B02.jp2 to ../data/tokyo/54-S-UE-2023-5-2/R10m/B02.jp2
download: s3://sentinel-s2-l2a/tiles/54/S/UE/2023/5/2/0/R10m/B04.jp2 to ../data/tokyo/54-S-UE-2023-5-2/R10m/B04.jp2
download: s3://sentinel-s2-l2a/tiles/54/S/UE/2023/5/2/0/R20m/AOT.jp2 to ../data/tokyo/54-S-UE-2023-5-2/R20m/AOT.jp2
download: s3://sentinel-s2-l2a/tiles/54/S/UE/2023/5/2/0/R10m/B03.jp2 to ../data/tokyo/54-S-UE-2023-5-2/R10m/B03.jp2
download: s3://sentinel-s2-l2a/tiles/54/S/UE/2023/5/2/0/R10m/TCI.jp2 to ../data/tokyo/54-S-UE-2023-5-2/R10m/TCI.jp2
download: s3://sentinel-s2-l2a/tiles/54/S/UE/2023/5/2/0/R10m/B08.jp2 to ../data/tokyo/54-S-UE-2023-5-2/R10m/B08.jp2
download: s3://sentinel-s2-l2a/tiles/54/S/UE/2023/5/2/0/R20m/B01.jp2 to ../data/tokyo/54-S-UE-2023-5-2/R20m/B01.jp2
download: s3://sentinel-s2-l2a/tiles/54/S/UE/2023/5/2/0

 20%|██        | 2/10 [12:33<52:22, 392.83s/it]

...

 80%|████████  | 8/10 [57:54<15:04, 452.13s/it]

54-S-UE-2023-2-16
download: s3://sentinel-s2-l2a/tiles/54/S/UE/2023/2/16/0/R10m/AOT.jp2 to ../data/tokyo/54-S-UE-2023-2-16/R10m/AOT.jp2
download: s3://sentinel-s2-l2a/tiles/54/S/UE/2023/2/16/0/R10m/B02.jp2 to ../data/tokyo/54-S-UE-2023-2-16/R10m/B02.jp2
download: s3://sentinel-s2-l2a/tiles/54/S/UE/2023/2/16/0/R10m/B03.jp2 to ../data/tokyo/54-S-UE-2023-2-16/R10m/B03.jp2
download: s3://sentinel-s2-l2a/tiles/54/S/UE/2023/2/16/0/R20m/AOT.jp2 to ../data/tokyo/54-S-UE-2023-2-16/R20m/AOT.jp2
download: s3://sentinel-s2-l2a/tiles/54/S/UE/2023/2/16/0/R10m/B04.jp2 to ../data/tokyo/54-S-UE-2023-2-16/R10m/B04.jp2
download: s3://sentinel-s2-l2a/tiles/54/S/UE/2023/2/16/0/R10m/B08.jp2 to ../data/tokyo/54-S-UE-2023-2-16/R10m/B08.jp2
download: s3://sentinel-s2-l2a/tiles/54/S/UE/2023/2/16/0/R10m/TCI.jp2 to ../data/tokyo/54-S-UE-2023-2-16/R10m/TCI.jp2
download: s3://sentinel-s2-l2a/tiles/54/S/UE/2023/2/16/0/R20m/B01.jp2 to ../data/tokyo/54-S-UE-2023-2-16/R20m/B01.jp2
download: s3://sentinel-s2-l2a/tiles/5

 90%|█████████ | 9/10 [1:04:56<07:22, 442.72s/it]

54-S-UE-2023-2-6
download: s3://sentinel-s2-l2a/tiles/54/S/UE/2023/2/6/0/R10m/AOT.jp2 to ../data/tokyo/54-S-UE-2023-2-6/R10m/AOT.jp2
download: s3://sentinel-s2-l2a/tiles/54/S/UE/2023/2/6/0/R10m/B03.jp2 to ../data/tokyo/54-S-UE-2023-2-6/R10m/B03.jp2
download: s3://sentinel-s2-l2a/tiles/54/S/UE/2023/2/6/0/R20m/AOT.jp2 to ../data/tokyo/54-S-UE-2023-2-6/R20m/AOT.jp2
download: s3://sentinel-s2-l2a/tiles/54/S/UE/2023/2/6/0/R10m/B02.jp2 to ../data/tokyo/54-S-UE-2023-2-6/R10m/B02.jp2
download: s3://sentinel-s2-l2a/tiles/54/S/UE/2023/2/6/0/R10m/B04.jp2 to ../data/tokyo/54-S-UE-2023-2-6/R10m/B04.jp2
download: s3://sentinel-s2-l2a/tiles/54/S/UE/2023/2/6/0/R10m/B08.jp2 to ../data/tokyo/54-S-UE-2023-2-6/R10m/B08.jp2
download: s3://sentinel-s2-l2a/tiles/54/S/UE/2023/2/6/0/R20m/B01.jp2 to ../data/tokyo/54-S-UE-2023-2-6/R20m/B01.jp2
download: s3://sentinel-s2-l2a/tiles/54/S/UE/2023/2/6/0/R10m/TCI.jp2 to ../data/tokyo/54-S-UE-2023-2-6/R10m/TCI.jp2
download: s3://sentinel-s2-l2a/tiles/54/S/UE/2023/2/6/0

100%|██████████| 10/10 [1:10:35<00:00, 423.58s/it]


## Loading data

In [6]:
BANDS = [
    'R20m/B01', 'R10m/B02', 'R10m/B03', 'R10m/B04', 'R20m/B05', 'R20m/B06', 
    'R20m/B07', 'R10m/B08', 'R20m/B8A', 'R60m/B09', 'R20m/B11', 'R20m/B12',
]

In [7]:
PATHS_NPY = []

for PATH_S3 in tqdm(PATHS_SENTILEL2_S3):
    SCENE_ID = PATH_S3.split('tiles')[1].replace('/', '-')[1:-3]
    print(SCENE_ID)
    OUTPUT_PATH = f'{cfg.PATH_DATA}/{SCENE_ID}/'
    
    imgs = []
    
    for BAND in BANDS:
        PATH_BAND = f'{OUTPUT_PATH}{BAND}.jp2'
        img = cv2.imread(PATH_BAND, cv2.IMREAD_UNCHANGED)        
        if 'R20m' in BAND:
            img = cv2.resize(img, (img.shape[1]*2, img.shape[0]*2), interpolation=1)
        elif 'R60m' in BAND:
            img = cv2.resize(img, (img.shape[1]*6, img.shape[0]*6), interpolation=1)

        imgs.append(img)
        
    imgs = np.stack(imgs, axis=2)
    
    for h_idx in range(imgs.shape[0]//512):
        for w_idx in range(imgs.shape[1]//512):
            
            patch = imgs[h_idx*512:h_idx*512+512, w_idx*512:w_idx*512+512, :]
                
            PATH_NPY = f'{cfg.PATH_NPY_ROOT}img/{SCENE_ID}_h{h_idx}_w{w_idx}.npy'
            np.save(PATH_NPY, patch)
            PATHS_NPY.append(PATH_NPY)


  0%|          | 0/10 [00:00<?, ?it/s]

54-S-UE-2023-5-17


 10%|█         | 1/10 [02:26<21:58, 146.47s/it]

54-S-UE-2023-5-2


 20%|██        | 2/10 [04:56<19:46, 148.35s/it]

...

 80%|████████  | 8/10 [22:55<06:22, 191.31s/it]

54-S-UE-2023-2-16


 90%|█████████ | 9/10 [27:02<03:28, 208.77s/it]

54-S-UE-2023-2-6


100%|██████████| 10/10 [29:45<00:00, 178.58s/it]


In [8]:
df = pd.DataFrame({
    'feature': PATHS_NPY
})

df['name'] = df['feature'].apply(lambda x: x.split('/')[-1].split('.')[0])
df['mask'] = False
df['defects'] = False
df['fold'] = -1
df['hight'] = 512
df['width'] = 512
df['california'] = False
df['cv'] = -1
df['area'] = 'tokyo'

df.head(4)

,feature,name,mask,defects,fold,hight,width,california,cv,area
0,../data/019_Tokyo/img/54-S-UE-2023-5-17_h0_w0.npy,54-S-UE-2023-5-17_h0_w0,False,False,-1,512,512,False,-1,tokyo
1,../data/019_Tokyo/img/54-S-UE-2023-5-17_h0_w1.npy,54-S-UE-2023-5-17_h0_w1,False,False,-1,512,512,False,-1,tokyo
2,../data/019_Tokyo/img/54-S-UE-2023-5-17_h0_w2.npy,54-S-UE-2023-5-17_h0_w2,False,False,-1,512,512,False,-1,tokyo
3,../data/019_Tokyo/img/54-S-UE-2023-5-17_h0_w3.npy,54-S-UE-2023-5-17_h0_w3,False,False,-1,512,512,False,-1,tokyo


In [9]:
df.to_csv(cfg.PATH_CSV, index=False, header=True)